In [18]:
import pandas as pd
import geopandas as gpd

In [19]:
cols = ['short_name', 'station_id', 'ntaname', 'ntacode']
df = pd.read_pickle('../temp/stations-with-nta.pickle')[cols]
df

,short_name,station_id,ntaname,ntacode
0,6926.01,72,Clinton,MN15
1,5430.08,79,SoHo-TriBeCa-Civic Center-Little Italy,MN24
2,5167.06,82,Chinatown,MN27
3,4354.07,83,Fort Greene,BK68
4,6148.02,116,Hudson Yards-Chelsea-Flatiron-Union Square,MN13
...,...,...,...,...
1620,4920.13,4748,Battery Park City-Lower Manhattan,MN25
1621,7034.08,4753,Old Astoria,QN71
1622,5329.08,4754,SoHo-TriBeCa-Civic Center-Little Italy,MN24
1623,8494.01,4755,Fordham South,BX40


In [20]:
rides = pd.read_pickle("../temp/merged.pickle")
rides['start_station_id'] = rides['start_station_id'].astype('str')


In [21]:
merged = rides.merge(df, left_on="start_station_id", right_on="short_name", how='left').sample(40000)
merged.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,uuid,short_name,station_id,ntaname,ntacode
1485149,3341A4AD0CB85AA3,electric_bike,2022-04-15 14:50:03,2022-04-15 15:06:37,Wythe Ave & Metropolitan Ave,5348.02,Clermont Ave & Park Ave,4692.01,40.716887,-73.963198,40.695734,-73.971297,member,b8cda51e-ddd5-11ec-b338-f6dfb783c623,5348.02,2002,North Side-South Side,BK73
1326228,1AD79872071C2812,classic_bike,2022-04-04 18:11:50,2022-04-04 18:16:23,Stuyvesant Walk & 1 Av Loop,5854.1,E 33 St & 1 Ave,6197.08,40.732340,-73.978984,40.743227,-73.974498,member,b8cda51e-ddd5-11ec-b338-f6dfb783c623,NaN,NaN,NaN,NaN
2005316,F4DF178B302F2FEB,docked_bike,2022-04-30 16:37:21,2022-04-30 17:05:51,Old Slip & South St,4993.13,Albany St & Greenwich St,5145.02,40.703367,-74.007868,40.709267,-74.013247,casual,b8cda51e-ddd5-11ec-b338-f6dfb783c623,4993.13,4248,Battery Park City-Lower Manhattan,MN25
729431,1DF8D10EF2DB1B23,electric_bike,2022-04-13 14:42:48,2022-04-13 14:52:48,Washington Square E,5755.09,Avenue C & E 18 St,5769.04,40.730494,-73.995721,40.730563,-73.973984,member,b8cda51e-ddd5-11ec-b338-f6dfb783c623,5755.09,294,West Village,MN23
105974,B4668683D6CED14F,classic_bike,2022-04-25 22:22:59,2022-04-25 22:32:34,W 11 St & 6 Ave,5914.03,E 9 St & Avenue C,5616.01,40.735324,-73.998004,40.725213,-73.977688,member,b8cda51e-ddd5-11ec-b338-f6dfb783c623,5914.03,254,West Village,MN23


In [39]:
merged['start_hour'] = merged['started_at'].dt.hour
# Resample time series by counting rides per every hour
df = (merged
  .set_index('started_at')
  .groupby(['short_name'])
  .resample('1H')
  .count()
  .loc[:, 'ride_id']
  .reset_index())
# Use a general hour field
df['start_hour'] = df['started_at'].dt.hour
df
# Group by the general hour field and count # of rides there
df.groupby(['short_name', 'start_hour']).sum()
# count makes sense as output for ride_id

ride_id
short_name start_hour         
2733.03    0                 0
           1                 0
           2                 0
           3                 0
           4                 0
...                        ...
8841.03    19                0
           20                0
           21                0
           22                0
           23                0

[33128 rows x 1 columns]

In [41]:
stations_per_nta = merged[['ntacode', 'start_station_id']].groupby('ntacode')[['start_station_id']].nunique().reset_index().set_index('ntacode').rename({'start_station_id': 'stations_count'}, axis=1)
stations_per_nta

,stations_count
ntacode,
BK09,13
BK31,4
BK32,36
BK33,34
BK34,10
...,...
QN68,19
QN70,43
QN71,11


In [42]:
# Given df of rides with NTA's joined, count the number of rides and rank them within each group.
def rank_stations_by_nta(df, df_stations_per_nta):
    stations_ranked_by_nta = (df[['uuid', 'ntacode', 'start_station_id']]
        .groupby(['ntacode', 'start_station_id'])
        .count()
        .sort_values(by=['ntacode', 'uuid'], ascending=False)
        .groupby(['ntacode'])
        .rank(method='dense', ascending=False, pct=False)
        .reset_index()
        .rename({'uuid': 'station_rank'}, axis=1)
        .merge(df_stations_per_nta, on='ntacode', how='left')
        .set_index('start_station_id')
    )
    stations_ranked_by_nta['station_rank'] = stations_ranked_by_nta['station_rank'].astype('int32')

    return stations_ranked_by_nta.reset_index()

rank_stations_by_nta(merged, stations_per_nta)

,start_station_id,ntacode,station_rank,stations_count
0,7159.08,QN99,1,3
1,7291.06,QN99,2,3
2,7271.01,QN99,3,3
3,7126.01,QN72,1,34
4,7144.01,QN72,2,34
...,...,...,...,...
1470,4756.04,BK09,8,13
1471,4645.04,BK09,9,13
1472,4718.08,BK09,9,13
1473,4829.01,BK09,10,13
